# Neuroimaging week 6: Group-level models, MCC, and ROI-analyses
This week's lab is about grouplevel models, multiple comparison correction (MCC), and region-of-interest (ROI) analysis. We will demonstrate these concepts using both FSL and Python. 

In part 1, we'll focus on the 'summary statistics' approach again, in which we'll demonstrate how we average $c\beta$-terms across runs (in run-level analyses) and subjects (in grouplevel analyses) using the GLM. Then, we're going to show you how to test more extensive hypotheses in grouplevel models. In part 2, we're going to show you the effect of different multiple comparison correction (MCC) techniques using real and simulated data. And in part 3, we'll introduce ROI-based analyes as an alternative to the whole-brain analyses we've dealt with so far.

### Contents
1. Group-level models
2. Multiple comparison correction
3. ROI-based analysis

### What you'll learn
Specifically, after this lab, you'll ...

- understand the concept of the summary statistics approach
- be able to construct different grouplevel models (in FSL)
- know the relative advantages and disadvantages of different MCC techniques
- understand and implement ROI-based analyses (in Python)

**Estimated time needed to complete**: 6 hours<br>
**Credits**: the MCC part of this notebook is based on a blog by [Matthew Brett](https://matthew-brett.github.io/teaching/random_fields.html) and a previous Matlab-based lab by H. Steven Scholte.<br>

<div class='alert alert-success'>
    <b>Tip!</b>
It's totally okay to skip a ToDo/ToThink/Assignment if you find it too hard or takes too much time! Almost always, the ToDos/ToThinks/assignments are independent of each other, so skipping one doesn't affect the others. The points you get for a single ToDo/ToThink are just a tiny fraction of all points, so skipping one does not affect your grade for your notebook a lot.
</div>

In [ ]:
# Some imports
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## 1. Group-level models
Last week, we discussed the multilevel models and the summary statistics approach. Specifically, we focused on how data from different runs are usually (in non-fMRI contexts) are analyzed in a single multilevel GLM by "concatenating" the data. And how, in fMRI, we usually don't take this approach due to the computational burden and use the summary statistics approach, which analyzes each run separately and subsequently aggregates the data in a second, run-level GLM. 

In this notebook, we will extend this idea of analyzing data from multiple levels by looking at data from multiple subjects and how to analyze this data in group-level models. We will look at two "flavors" of group-level analyses: parametric and non-parametric.

![](https://docs.google.com/drawings/d/e/2PACX-1vQxCH3WU3nTqFlHUZb49rf9zioivGQ-flVfRpwmXQx7OF5Wm_1T6gFMYQqpqt-NPITNHUaRoVYEREgT/pub?w=965&h=745)

### 1.1. Parametric analyses
The most often used "flavor" of fMRI (group-level) analyses are *parametric*: it assumes that the data can be modeled using specific probability distributions. For example, in the GLM, we assume that the results of statistical tests of parameters (i.e., $t$-values) are distributed according to the Students $t$-distribution (with a particular degrees-of-freedom):

\begin{align}
t_{c\hat{\beta}} \sim \mathcal{T}(\mathrm{df})
\end{align}

where you can read the $\sim$ symbol as "is distributed as". Importantly, the validity of the computed $p$-values depends on whether the choice of distribution is appropriate. If not, you might risk inflated type 1 or type 2 errors.

The first-level and run-level GLMs that we have discussed so far are examples of parametric analyses. There are also *non-parametric* versions of the GLM that do not assume any particular form of distribution; while somewhat more computationally expensive, this is become a more and more popular alternative to (group-level) parametric analyses. Importantly, the difference between parametric and non-parametric analyses is only important for the *inference* (not the *estimation*) aspect of the (group-level) analyses.

Now, let's focus on the parametric version of group-level analyses. Basically, this amounts to doing the same thing as we did last week with the run-level analyses, but this time, the results from our run-level analyses ($c\hat{\beta}^{*}$) across different subjects will become our target ($y^{\dagger}$). Note that we will use the "dagger" ($^{\dagger}$) superscript to denote that the mathematical terms belong to the group-level model (just like the $^{*}$ superscript in our notebooks refers to terms belonging to the run-level models).

To reiterate, the results from our run-level analyses ($c\hat{\beta}^{*}$), or first-level analyses if we only have a single run, become our dependent variable in our group-level analysis ($y^{\dagger}$):

\begin{align}
y^{\dagger} = c\hat{\beta}^{*}
\end{align}

Again, the group-level represents a GLM with a particular design matrix ($\mathbf{X}^{\dagger}$) and parameters ($\beta^{\dagger}$):

\begin{align}
y^{\dagger} = \mathbf{X}^{\dagger}\beta^{\dagger} + \epsilon^{\dagger}
\end{align}

And the group-level parameters can be estimated with OLS:

\begin{align}
\hat{\beta}^{\dagger} = (\mathbf{X}^{\dagger\ T} \mathbf{X}^{\dagger})^{-1}\mathbf{X}^{\dagger}y^{\dagger}
\end{align}

As mentioned last week, the parameter estimation procedure (i.e., estimating $\hat{\beta}^{\dagger}$) is relatively straightforward, but the inference procedure depends on the specific variance approach: fixed, random, or mixed effects. If your aim is to perform inference to the population, *you should never use a fixed-effects type of analysis across subjects*, as this will typically inflate your type 1 error greatly.

That leaves us with random effects and mixed effects GLMs. 

#### 1.1.1. Random effects
Let's first focus on a random effects type of analysis (which is commonly used in group-level analysis in the SPM software package, for example), as this is relatively easy to understand. For now, let's use simulated data. Suppose that we want to do a group-analysis of our run-level `flocBLOCKED` data. Specifically, we are interested in the difference between faces and places ($H_{0}: \beta_{face}^{*} = \beta_{house}^{*} = 0$, $H_{a}: \beta_{face}^{*} > \beta_{house}^{*}$). As such, we'll use the corresponding $c\hat{\beta}^{*}$ terms from our run-level analysis (i.e., the contrasts against baseline, COPE1 and COPE2) as our new target $y^{\dagger}$ as follows:

\begin{align}
y^{\dagger} = \begin{bmatrix}
c\hat{\beta}^{*}_{1, F>0} \\
c\hat{\beta}^{*}_{2, F>0} \\
\vdots \\
c\hat{\beta}^{*}_{N, F>0} \\
c\hat{\beta}^{*}_{1, P>0} \\
c\hat{\beta}^{*}_{2, P>0} \\
\vdots \\
c\hat{\beta}^{*}_{N, P>0}
\end{bmatrix}
\end{align}

For our simulation, we'll assume that we have 20 subjects ($N=20$).

<div class='alert alert-info'>
    <b>ToThink</b> (0 points)
</div>

You might think, why compute the difference contrast at the *group-level*, while we could have computed this already in the first-level analysis (and subsequently average this in the run-level and group-level analyses)? In fact, this is mathematically (largely) equivalent. However, most people prefer to postpone this step to the group-level analysis. Can you think of a reason why?

Before we are going to simulate the data, let's construct our design-matrix, $\mathbf{X}^{*}$. Assuming we have 20 subjects and two types of inputs ($c\hat{\beta}^{*}_{F>0}$ and $c\hat{\beta}^{*}_{P>0}$, our design matrix will be:

In [ ]:
N = 20
X = np.zeros((N * 2, 2))
X[:N, 0] = 1
X[N:, 1] = 1

print(X)

<div class='alert alert-info'>
    <b>ToThink</b> (1 point)
</div>

In the above design matrix, we did not specify an intercept (i.e., a vector of ones). With good reason, because our group-level GLM will crash if we did. Explain shortly why an intercept for this particular design matrix (i.e., as represented by the variable `X` above) is not only redundant, but will make the GLM impossible to estimate. Hint: try adding an intercept, run a GLM, and see what happens.

YOUR ANSWER HERE

We are going to simulate data for a single voxel, which shows a slight preference (in run-level $c\hat{\beta}^{*}$ estimates across subjects) for faces relative to places. Specifically, we'll assume that the voxel activates on average with $\beta^{\dagger}_{faces} = 5$ for faces, while it activates on average with $\beta^{\dagger}_{places} = 4.5$ for places:

\begin{align}
y^{*} = \mathbf{X}^{\dagger}_{faces}\cdot 5 + \mathbf{X}^{\dagger}_{places} \cdot 4 + \epsilon^{\dagger} 
\end{align}

where the noise, $\epsilon^{\dagger}$, is normally distributed with mean 0 and a standard deviation of 1:

\begin{align}
\epsilon^{\dagger} \sim \mathcal{N}(0, 1)
\end{align}

(the symbol $\sim$ stands for "is distributed as".) We'll implement this in code below:

In [ ]:
N = 20
beta_f = 5
beta_p = 4.5
std_noise = 1
np.random.seed(42)
noise = np.random.normal(0, std_noise, size=N * 2)

y_sim = X @ np.array([beta_f, beta_p]) + noise

And let's plot it:

In [ ]:
plt.figure(figsize=(5, 16))
plt.plot(y_sim[:N], np.arange(y_sim.size // 2), marker='o', ms=10)
plt.plot(y_sim[N:], np.arange(y_sim.size // 2, y_sim.size), marker='o', ms=10)
plt.ylim(y_sim.size, -1)
plt.legend([r"$c\hat{\beta}^{*}_{face}$", r"$c\hat{\beta}^{*}_{place}$"], fontsize=15)
plt.ylabel("Subjects", fontsize=20)
plt.xlabel(r"$c\hat{\beta}^{*}$", fontsize=20)
plt.grid()
plt.title(r"$y^{\dagger}$ (run-level estimates)", fontsize=25)
plt.show()

Note that these $c\hat{\beta}^{*}$ estimates are the only thing you need for a random effects analysis! We're simply ignoring the lower-level variance terms.

<div class='alert alert-warning'>
    <b>ToDo</b> (2 points): Run a group-level random-effects GLM (using the variable <tt>X</tt> as design matrix and <tt>y_sim</tt> as dependent variable). Store the estimated parameters in a variable named <tt>gl_params</tt>. Then, compute the $t$-value for the contrast $\beta^{\dagger}_{face} > \beta^{\dagger}_{place}$ and store this in a variable named <tt>gl_tvalue</tt>.
</div>

In [ ]:
''' Implement the ToDo here. '''
from numpy.linalg import inv

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
''' Tests the above ToDo. '''
from niedu.tests.nii.week_6 import test_rfx_glm
test_rfx_glm(X, y_sim, gl_params, gl_tvalue)

#### 1.1.2. Mixed-effects (in FSL)
Proper mixed-effects models are a bit too complex to implement ourselves in Python, so we'll show you how to do it in FSL! We ran the (fixed-effects) run-level models for the `flocBLOCKED` data for you already. Like last week, the analysis results are stored in the following directory:

In [ ]:
import os
from niedu.global_utils import get_data_dir

data_dir = get_data_dir()
fsl_deriv_dir = os.path.join(data_dir, 'derivatives', 'fsl')
print("Data is in %s" % fsl_deriv_dir)

Each subject has three subdirectories (`flocER`, `flocBLOCKED`, and `face`) with their first-level and run-level results. The run-level results for each subject are stored in a directory named `runlevel.gfeat`, with the following contents:

<div class='alert alert-warning'>
    <b>ToDo</b> (ungraded)
    
Start a remote desktop environment, open a terminal, navigate to the `runlevel.gfeat` directory of the task `flocBLOCKED` of subject `sub-03` (so `sub-03/flocBLOCKED/runlevel.gfeat`), and print the directory's contents to the terminal.

(Note that the following ToDos will be checked later using test-cells.)
</div>

You should see that the `runlevel.gfeat` directory contains multiple `cope*.feat` directories. These directories correspond to the results of the run-level (intercept-only) analyses of the seven different first-level contrasts. In contrast to what you did yourself last week (in which you concatenated the COPEs from the $c\hat{\beta}_{face}$ and $c\hat{\beta}_{place}$ contrasts in a single GLM), we were lazy and used the "Inputs are lower-level FEAT directories" option in FEAT. This runs the exact same GLM (using an intercept-only run-level design matrix) for all first-level contrasts separately. 

For example, the data in the `runlevel.gfeat/cope1.feat` directory corresponds to the first contrast from the first-level analysis ($\beta_{face} > 0$), the data in the `runlevel.gfeat/cope2.feat` directory corresponds to the second contrast from the first-level analysis ($\beta_{place} > 0$), etc. etc.

In our intercept-only run-level analysis, we only computed a run-level contrast reflecting the average across the two sessions (i.e., using the contrast vector $[1]$). The results of this run-level contrast, for each first-level contrast, are stored in the `cope1.nii.gz` file.

<div class='alert alert-warning'>
    <b>ToDo</b> (ungraded)
</div>

Open Feat, change "First-level analysis" to "Higher-level analysis" and change the input type to "Inputs are 3D cope images from feat directories". Then, click on the "Stats" tab.

Under the "Stats" tab, you can change the type of inference you want FEAT to do. By default, it uses a particular mixed-effects approach that FSL calls "FLAME 1". In general, we recommend using this type of inference. The other mixed-effects approach, "FLAME 1+2", is also a good choice, but may take much longer to run. The "Mixed effects: Simple OLS" is, unlike the same suggests, the random-effects option, which discards the lower-level variance estimates. Lastly, the "Randomise" option is a non-parametric (random-effects) variant, that will be discussed in more detail later.



<div class='alert alert-warning'>
    <b>ToDo</b> (ungraded)
</div>

Set the option to "Mixed effects: FLAME 1". Enable the "use automatic outlier de-weighting" option (should be yellow). Then, go to post-stats and set the "Thresholding" option to "Cluster" with a Z threshold of 3.7 and a "Cluster P threshold" of 0.05. This refers to a particular type of multiple comparison correction that will be discussed later.

Now, let's define the dependent variable ($y^{\dagger}$). Suppose, just like we did with the simulated data, that we have the following null and alternative hypothesis:

$H_{0}: \beta_{face}^{*} - \beta_{house}^{*} = 0$<br>
$H_{a}: \beta_{face}^{*} - \beta_{house}^{*} > 0$

Note that the contrast evaluating the difference between these two parameters was *not* computed in the first-level or run-level analyses, so we have to create this ourselves by concatenating the COPEs corresponding to the $\beta_{face}^{*} > 0$ and $\beta_{house}^{*} > 0$ run-level contrasts, just like you did last week. 

Doing this for all subjects becomes a bit cumbersome though, so you only need to do this for `sub-02`, `sub-03`, and `sub-04`. Make sure the first three inputs correspond to the face-against-baseline COPE (COPE nr. 1) and the last three inputs correspond to the place-against-baseline COPE (COPE nr. 2). 

\begin{align}
y^{\dagger} = \begin{bmatrix}
c\hat{\beta}^{*}_{\mathrm{face>0,\ sub-02}} \\
c\hat{\beta}^{*}_{\mathrm{face>0,\ sub-03}} \\
c\hat{\beta}^{*}_{\mathrm{face>0,\ sub-04}} \\
c\hat{\beta}^{*}_{\mathrm{place>0,\ sub-02}} \\
c\hat{\beta}^{*}_{\mathrm{place>0,\ sub-03}} \\
c\hat{\beta}^{*}_{\mathrm{place>0,\ sub-04}} \\
\end{bmatrix}
\end{align}

Note that you need the actual `cope*.nii.gz` files &mdash; *not* the `.feat` directory itself.

(Realize, though,  that having only 3 subjects for your group-analysis would be a hopelessly underpowered study.)

<div class='alert alert-warning'>
    <b>ToDo</b> (1 point)
</div>

In the "Data" tab, use the option "Inputs are 3D cope images from FEAT directories", set the number of inputs to 6, and select the correct files. Note that within the "Select input data" window, you can copy/paste the paths using CTRL+C / CTRL+V, which might make the process a little easier/faster (not forget to actually modify the paths after copy/pasting).

<div class='alert alert-warning'>
    <b>ToDo</b> (ungraded)
</div>

In the "Data" tab, set the output directory to your home folder + `grouplevel_task-flockBLOCKED`.

<div class='alert alert-warning'>
    <b>ToDo</b> (1 point)
</div>

In the “Stats” tab, click on "Full model setup". Create two predictors ("EVs") that model the mean of the $c\beta^{*}_{face} > 0$ (first EV) and the $c\beta^{*}_{place} > 0$ (second EV). Then, add another predictor that models the effect of age on the $c\beta^{*}_{face}$ contrast *only*. The ages of the three participants (from sub-02 to sub-04) are: $[21, 32, 25]$. Importantly, make sure you center (de-mean) the age values before entering them in FEAT, as this improves the interpretation of the other predictors (you can compute the de-meaned age values, for example, in a new code cell).

<div class='alert alert-warning'>
    <b>ToDo</b> (1 point)
</div>

In the "Contrasts & F-tests" tab, define the following group-level contrasts:
1. $H_{0}: \beta^{\dagger}_{face} = 0$<br>$H_{a}: \beta^{\dagger}_{face} > 0$<br><br>
2. $H_{0}: \beta^{\dagger}_{place} = 0$<br>$H_{a}: \beta^{\dagger}_{place} > 0$<br><br>
3. $H_{0}: \beta^{\dagger}_{face} - \beta^{\dagger}_{place} = 0$<br>$H_{a}: \beta^{\dagger}_{face} - \beta^{\dagger}_{place} > 0$<br><br>
4. $H_{0}: \beta^{\dagger}_{age} = 0$<br>$H_{a}: \beta^{\dagger}_{age} > 0$<br><br>
5. $H_{0}: \beta^{\dagger}_{age} = 0$<br>$H_{a}: \beta^{\dagger}_{age} < 0$<br><br>


<div class='alert alert-warning'>
    <b>ToDo</b> (ungraded)
</div>

Save your design (using the "Save" button in the bottom of the FEAT window) in your `week_6` directory and name it `setup_feat_grouplevel.fsf`.

<div class='alert alert-danger'>
    <b>For UvA students: do not actually run this group-level analysis!</b>
     Having a large group of students run a group-level analysis may or may not crash the server; let's not risk it. 
</div>

Below, you can test your implementation.

In [ ]:
''' Tests your setup_feat.fsf file: inputs'''
import os
import os.path as op
import niedu

here = op.abspath('')
par_dir = op.basename(op.dirname(here))
home = op.expanduser("~")

if par_dir != 'source':  # must be a student-account
    fsf = 'setup_feat_grouplevel.fsf'
    if not op.isfile(fsf):
        print("Couldn't find a 'setup_feat_group.fsf' file in your week_6 directory!")

    fsl_deriv_dir_tmp = fsl_deriv_dir
    with open(fsf, 'r') as f:
        fsf = f.readlines()

    fsf = [txt.replace('\n', '') for txt in fsf if txt != '\n']
    fsf = [txt for txt in fsf if txt[0] != '#']  # remove commnts

    from niedu.tests.nii.week_6 import test_grouplevel_fsf_inputs
    test_grouplevel_fsf_inputs(fsf, fsl_deriv_dir_tmp)

In [ ]:
''' Tests your setup_feat.fsf file: Evs'''
if par_dir != 'source':
    from niedu.tests.nii.week_6 import test_grouplevel_fsf_evs
    test_grouplevel_fsf_evs(fsf)

In [ ]:
''' Tests your setup_feat.fsf file: contrasts'''
if par_dir != 'source':
    from niedu.tests.nii.week_6 import test_grouplevel_fsf_cons
    test_grouplevel_fsf_cons(fsf)

### 1.2. Non-parametric analyses
*This is an optional section*

Before we go on with the topic of "multiple comparison correction", let's focus on another type of inference in statistical models: permutation-based non-parametric analyses. Unlike parametric analyses (which we discussed in the previous section), non-parametric analyses do not assume anything about how our statistics-of-interest are distributed. Instead, it uses the data itself to estimate a distribution of your test-statistic under the null hypothesis. Note that this type of inference is also available in FSL, where it is called "randomise". 

Permutation tests work (roughly) as follows:
1. The statistic (e.g., $t$-value) is computed as normal;
2. The rows of design matrix ($\mathbf{X}$) are shuffled and the statistic is (re-)computed, which is iterated a number of times (e.g., 1000);
3. The ("non-parametric") $p$-value is computed

Let's do this for our previously simulated data (the variables `X` and `y_sim`).

#### 1.2.1. Computing the "observed" statistic
First, we need to compute our "observed" statistic. In the framework of the GLM, this is often a $t$-statistic (or $F$-statistic). But note that permutation-based tests work for any kind of statistics. (In fact, permutation-based tests are especially useful for statistics with unknown distributions!)

We are sure that you, by now, know how to compute $t$-values, so we are going to use a predefined function `compute_tvalue` with three arguments: `X`, `y`, and `c`. We'll compute the $t$-value for our data (`X` and `y_sim`) and store it in a variable called `t_obs` (for "observed").

In [ ]:
from niedu.utils.nii import compute_tvalue
t_obs = compute_tvalue(X, y_sim, c=np.array([1, -1]))

#### 1.2.2. Run the permutations
The second step is to, iteratively, permute the data and (re-)compute the test statistic. In most cases (assuming that the datapoints, $y$, are completely independent, i.e., there is no "autocorrelation"), permutation within the GLM framework is done by shuffling the rows of the design matrix ($\mathbf{X}$). This is usually done a larger number of times (e.g., 1000 or more).

<div class='alert alert-warning'>
    <b>ToDo</b> (ungraded)
</div>

Below, we started implementing the permutation loop. Can you finish it? You may want to use the [np.random.permutation](https://docs.scipy.org/doc/numpy-1.14.0/reference/generated/numpy.random.permutation.html) function. Make sure to save the permuted t-value each iteration (by storing it in the `t_perm` array).

In [ ]:
# Implement your ToDo here

np.random.seed(42)
iters = 1000
t_perm = np.zeros(iters)

for i in range(iters):
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
''' Tests the ToDo above. '''
np.testing.assert_almost_equal(t_perm.mean(), 0, decimal=1)
print("Well done!")

#### 1.2.3. Calculate the p-value
The last step is to calculate the $p$-value. This is done by dividing the number of permuted statistics *larger* than the observed statistic plus one by the total number of permutations plus one. Or, mathematically, for $P$ permutations, the $p$-value for any statistic $s$ with observed value $s^{obs}$ and vector of permuted values $\mathbf{s}^{perm}$ is defined as:

\begin{align}
p = \frac{\sum_{i=1}^{P}\mathbf{I}(\mathbf{s}^{perm}_{i} \geq s^{obs}) + 1}{P + 1}
\end{align}

where $\mathbf{I}$ is an indicator function, returning $1$ when it condition is true and $0$ otherwise.

We can also visualize it:

In [ ]:
plt.figure(figsize=(10, 5))
plt.hist(t_perm)
plt.axvline(t_obs, ls='--', c='red', lw=3)
plt.legend(['observed', 'permuted'])
plt.xlabel(r'$t$-value', fontsize=20)
plt.ylabel('Frequency', fontsize=20)
plt.show()

In the above figure, the number of values right of the dashed red line ($t^{obs}$) plus 1 divided by the number of permutations plus 1 ($P+1$) is our $p$-value! Now, try computing it yourself!

<div class='alert alert-warning'>
    <b>ToDo</b> (ungraded)
</div>

Compute the $p$-value corresponding to our observed $t$-statistic (`t_obs`) and store it in a variable named `p_perm`.

In [ ]:
# Implement your ToDo here
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
''' Tests the above ToDo. 
Note: this may differ slightly due to the exact implementation of the permutation procedure
'''
np.testing.assert_almost_equal(p_perm, 0.03, decimal=2)
print("Well done!")

<div class='alert alert-warning'>
    <b>ToDo</b> (ungraded)
</div>

Shuffling rows works for most designs, but what if you have an intercept-only model (i.e., a one-sample $t$-test), where a design matrix consisting only of a vector of ones? Permuting the rows won't work there. An alternative for this scenario is "sign flipping": randomly changing ones to minus ones (with replacement).

Below, we simulate some new data (`y_sim2` and `X2`). Can you compute the non-parametric $p$-value for this one-sample $t$-test? Use a 1000 permutations and store the $p$-value in a new variable named `p_perm2`.

Hint: to generate random "signs", take a look [here](https://stackoverflow.com/questions/46820182/randomly-generate-1-or-1-positive-or-negative-integer).

In [ ]:
np.random.seed(17)
y_sim2 = np.random.normal(0, 1, size=100)
X2 = np.ones(100)[:, np.newaxis]

iters = 1000

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
''' Tests the above ToDo. '''
# The p-value might slightly differ from 0.17 depending on the
# exact method of generating random signs.
np.testing.assert_almost_equal(p_perm2, 0.17, decimal=2)
print("Well done!")

#### 1.2.4. Threshold-Free Cluster Enhancement (TFCE)
Apart from (or perhaps because of) the fact that non-parametric analyses are blind to the distribution of effects, non-parametric analyses have an additional advantage: they can use a "trick" called "Threshold-Free Cluster Enhancement" (TFCE). This method allows for voxel-based inference while taking into account the spatial size of the effect. You could see TFCE as a method to "boost" high-amplitude voxels that are located within a cluster of other high-amplitude voxels. 

However, we don't know how these "boosted" values are distributed (they might or might not be $\mathcal{T}$ distributed)! Fortunately, as said before, non-parameteric analyses don't care how the computed statistic is distributed (under the null). How TFCE works mathematically is beyond the scope of the course (but you can learn more in [this video](https://www.youtube.com/watch?v=q7cWw8WC0Ws)), but in fact recommend using this technique in combination with FSL's `randomise` tool, also because it provides an elegant and effective way to control for multiple comparisons (the topic of section 3).

### 1.3.  Checking out results from group-level FEAT analyses
We have run a group-level model for a couple of our lower-level contrasts. The results from the group-level analysis that we are going to take a look at is in the following directory:

In [ ]:
gl_dir = os.path.join(fsl_deriv_dir, 'grouplevel_task-flocBLOCKED', 'contrast-faceGTother_method-FLAME1_thresh-uncorr05.gfeat')
print(gl_dir)

This contains the results from the group-level analysis of the lower-level $4 \cdot \beta_{face} - \beta_{place} - \beta_{body} - \beta_{character} - \beta_{object} > 0$ contrast. (The "faceGTother" stands for "face greater than other conditions".)

<div class='alert alert-warning'>
    <b>ToDo</b> (ungraded)
</div>

Open a terminal, navigate to the above directory, and print its contents to the terminal window.

You should see the usual files: HTML-reports, thresholded results (as nifti files), and files with information about the design (`design.con`, `design.fsf`, etc.). 

<div class='alert alert-warning'>
    <b>ToDo</b> (ungraded)
</div>

Open the `report.html` file (with the command `firefox report.html`). Click on "Inputs", which will show you the data that we used as inputs (i.e., $y^{\dagger}$) for our group-level analysis.

As you can see, the inputs for our group-level analysis are not individual `cope*.nii.gz` files, but entire `.feat` directories. This is because we used the "Inputs are lower-level FEAT directories" option.

<div class='alert alert-warning'>
    <b>ToDo</b> (ungraded)
</div>

Click on "Results". This will show you a page with the simple "intercept-only" design (i.e., a design for a one-sample $t$-test). Now, click on "Lower-level contrast 1 (average)" to view in the group-level result from this particular run-level contrast.

On this page, you can view the results, as $z$-statistic images, from the group-level analysis plotted on a set of axial slices. Below the brain plot, you can see the "time series plot", which does not actually show a "time series", but rather the input (i.e., $y^{\dagger}$) of the voxel with the highest $z$-value. Thus, the "time series" here do not reflect datapoints over time (as in first-level analyses), but run-level $c\hat{\beta}^{*}$ estimates of different subjects!

<div class='alert alert-warning'>
    <b>ToDo</b> (ungraded)
</div>

In your terminal, open `FSLeyes`. Then, click on `File` &rarr; `Add standard` &rarr; select `MNI152_T1_2mm_brain.nii.gz` &rarr; click on `Open`.

Then, click on `File` &rarr; `Add from directory` &rarr; navigate to and select the `contrast-faceGTothers_method-FLAME1_thresh-uncorr05.gfeat/cope1.feat` directory &rarr; click `Open`.

This automatically opens the `filtered_func_data.nii.gz` file: a 4D nifti file with the run-level $c\hat{\beta}^{*}$ estimates of different subjects across the fourth dimension (i.e., data from different subjects are represented by different volumes, our new $y^{\dagger}$). 

<div class='alert alert-warning'>
    <b>ToDo</b> (ungraded)
</div>

Set the minimum value to 100 (in the top menu) and change the colormap to "Red-Yellow". Note that the minimum value of 100 is rather arbitrary, but it removes some of the "visual clutter".

Now, go to voxel $[26, 39, 25]$, which is (roughly) located in the right fusiform gyrus (the "FFA") and, *for this participant* (`sub-02`), seems to activate relatively strongly in response to faces (rather than to other images). 

<div class='alert alert-warning'>
    <b>ToDo</b> (ungraded)
</div>

Increase the volume number from 0 to 1 (in the "Location" panel), which will show you the run-level $c\hat{\beta}^{*}$ estimates from a different subject (i.e., `sub-03`). You'll notice that this participant's "FFA" is in a slightly different location! In fact, when you view the other volumes (i.e., the $c\hat{\beta}^{*}$ maps of other subjects), you see that the anatomical location of the effects varies substantially!

<div class='alert alert-info'>
    <b>ToThink</b> (1 point)
</div>

Name two reasons why an effect (i.e., a significant "blob") may differ in anatomical location across subjects.

YOUR ANSWER HERE

Alright, now let's look at some real group-level results in FSLeyes!

<div class='alert alert-warning'>
    <b>ToDo</b> (ungraded)
</div>

In FSLeyes, delete the `cope1: filtered_func_data` overlay (or make it invisible by clicking on the eye icon). Then,
open the `thresh_zstat1.nii.gz` (`File` &rarr; `Add from file`) and set the colormap to "Red-Yellow".

Here, you see the "thresholded" $z$-values plotted on the template (MNI152, 2mm) that FSL by default uses for its group-analyses. 

<div class='alert alert-warning'>
    <b>ToDo/ToThink</b> (1 point)
</div>

Go to the voxel at location $[37, 60, 28]$. According to the Harvard-Oxford Subcortical structural atlas, to what anatomical brain region does this voxel most likely belong? Write this down in the text cell below. Check out the notebook from last week if you forgot how to open the Atlas panel.

YOUR ANSWER HERE

The red-yellow voxels that you see are those that have $z$-values corresponding to $p$-values smaller than 0.05 (the other voxels with $p$ > 0.05 are set to 0). This particular thresholding procedure gives what people often refer to as "uncorrected results". Although they are thresholded at a particular $p$-value, the results are referred to as "uncorrected" because they have not been corrected for multiple comparisons &mdash; the topic of the next section.

## 2. Multiple comparison correction (MCC)
Univariate analyses of fMRI data essentially test hypotheses about your data (operationalized as contrasts between your $\hat{\beta}$ estimates) *for each voxel* separately. So, in practice, given that the MNI (2 mm) standard template brain contains about 260,000 voxels, you're conducting 260,000 different statistical tests! The obvious problem, here, is that some tests might turn out significant, while they in fact do not contain any (task-related) activity: the result is driven just by chance.

As a researcher, you should strive to "filter out" the results which are driven by noise (*false positives*) and keep the results which are actually driven by the true effect (*true positives*) as much as possible. It turns out that the more tests you do, the larger the chance is that you will find one or more *false positives*. To deal with this, researchers often employ techniques for *multiple comparison correction* (MCC): **correcting** for the inflated chance of false positives when you have **multiple** tests (**comparisons**).

In this tutorial, we will walk you through an example (simulated) dataset on which different MCC techniques are employed. We'll focus on how these different techniques influence the chance for finding false positives.

### 2.1. The example
We'll work with the (simulated) group-level results of a hypothetical fMRI experiment. Suppose that the subjects in our hypothetical experiment were shown pictures of cats in the scanner, because we (the experimenters) were interested in which voxels would (de)activate significantly in reponse to these cat pictures (i.e. a contrast of the cat-picture-condition against baseline).

An example of an image shown to the subjects:
![test](cute_cat.jpeg)

After extensive preprocessing, we fitted first-level models in which we evaluated the cat-against-baseline contrast, in which the $t$-statistic refers to how strongly each voxel responded to the pictures of cats. After doing a proper group-level analysis, we now have a group-level $t$-statistic map, reflecting whether voxels on average (de)activated in response to the pictures of cats.  

<div class='alert alert-info'>
<b>ToThink</b> (0.5 point)
</div>

On average, what group-level $t$-statistic would you (approximately) expect to find if, in fact, there would be no voxel which reliably (de)activated in response to the cat-pictures?

YOUR ANSWER HERE

### 2.2. The data
Usually, your whole-brain group-level results are 3D $z$- or $t$-statictic maps of the size of a standard brain (usually the MNI 2mm template, which has about 260,000 voxels). Plotting in 3D, however, is incredibly cumbersome, so for the sake of the example, we'll assume that our group-level results are represented as a 2D $z$-statistic map, with dimensions $200 \times 200$. So, we'll pretend we analyzed the results based on a 2D brain with $200 \times 200$ "voxels". 

Because we work with simulated data, we can actually specify the "true effect". In reality, we never know this of course! We are going to assume that there is a small "blob" of voxels in the middle of our "brain" that activates reliably to pictures of cats (with a $z$-value of 5.5). This blob is therefore the true effect in our simulation. 

Let's first simulate the data.

In [ ]:
# You don't have to understand how this simulation works exactly
k = 200  # number of vox in each dimension
signal = np.zeros((k, k))
r = 10  # middle of the image
a, b = k // 2, k // 2  # width and height of the circle
y, x = np.ogrid[-a:k-a, -b:k-b]
mask = x * x + y * y <= r * r
signal[mask] = 5.5  # amplitude of effect!

print("Shape of statistic map: %s" % (signal.shape,))

Alright, now let's plot the true effect as a 2D image. We'll define a custom function for this, `plot_sim_brain`, to save us some work later.

In [ ]:
def plot_sim_brain(brain, mask=None, vmin=-7, vmax=7, cmap='seismic', title='', label='Z-value'):
    """ Plots an image of a simulated 2D 'brain' with statistic values, which may be 'masked'.
    
    Parameters
    ----------
    brain : numpy array
        A 2D numpy array with statistics
    mask : numpy array (or None)
        A 2D numpy array with booleans (True = do plot, False = do not plot). If None,
        the 'brain' is not masked.
    vmin : int/float
        Minimum value of colorbar
    vmax : int/float
        Maximum value of colorbar
    cmap : str
        Name of colormap to use
    title : str
        Title of plot
    label : str
        Label for colorbar
    """
    
    brainm = brain.copy()
    if mask is not None:  # threshold!
        brainm[~mask] = 0

    plt.figure(figsize=(8, 10))
    plt.imshow(brainm, vmin=vmin, vmax=vmax, aspect='auto', cmap=cmap)
    plt.axis('off')
    plt.title(title, fontsize=25)
    
    cb = plt.colorbar(orientation='horizontal', pad=0.05)
    cb.set_label(label, fontsize=20)
    plt.show()
    
plot_sim_brain(signal, title="True effect")

Now, due to the inherent spatial smoothness of fMRI, this particular manifestation of the effect is not very realistic. In particular, the sharp "edges" of the effect are unlikely to occur in real fMRI data. Therefore, to make it a little more realistic, we can spatially smooth the "true effect map"! We will use the `gaussian_filter` function (from `scipy.ndimage`) with a FWHM of 12 "voxels". 

In [ ]:
from scipy.ndimage import gaussian_filter

fwhm = 12
# Convert FWHM to sigma
sigma = fwhm / np.sqrt(8 * np.log(2))
signal_smooth = gaussian_filter(signal, sigma=sigma)
plot_sim_brain(signal_smooth, title="True effect (smooth)")

As you've learned in the past weeks, the chances are very slim that you'll find such a "crisp" (true) effect as shown above; often, you might observe significant voxels that are not driven by a true effect, but by (spurious) noise, reflecting false positives.

So, let's make our data a little more realistic by simulating some random noise, sampled from a normal distribution with mean 0 and a standard deviation of 1. Importantly, we are also going to smooth our noise with the same gaussian filter (with FWHM = 12): 

In [ ]:
np.random.seed(2)  # for reproducibility
noise = np.random.normal(0, 1, size=signal.shape)
noise = gaussian_filter(noise, sigma=sigma)
noise = noise / noise.std()

plot_sim_brain(noise, title='The noise')

Now, to complete our simulation, we'll simply add the signal and the noise together (we'll call this variable `data`).

In [ ]:
data = signal_smooth + noise
plot_sim_brain(data, title='The data!')

The plot above now represents our simulated data, which contains both a true signal (the "blob" in the middle) and some (spatially correlated) noise. As a researcher, you aim to threshold your data in such a way that you maximize the chance of finding your true signal (true positive effects) and minimize the chance of erroneously treating noise as significant effects (false positive effects).

### 2.3. Uncorrected statistics maps
In the early days of fMRI analyses, the extent of the MCC problem (more tests = more false positives) was not yet widely known. What researchers simply did was to calculate the $p$-values corresponding to the $z$-value (or $t$-value) maps and threshold those $p$-values using some fixed cutoff ("alpha value"), usually 0.05 or 0.01. 

To implement this, we can convert all our $z$-values to $p$-values, compute a "mask" (i.e., an array with `True` and `False` values, indicating which "voxels" survive the threshold and which do not), and set all "voxels" outside the mask to 0. 

Let's choose a significance level ($\alpha$) of 0.05.

In [ ]:
alpha = 0.05

Now, let's convert the $z$-values (in the variable `data`) to $p$-values. We'll use the `stats.norm.sf` function from the `scipy` package for this. (This is the same type of function &mdash; a "survival function" &mdash; that we used to calculate the $p$-values corresponding to $t$-values before, but this time we use it for $z$-values)

In [ ]:
# This line converts the z-values to p-values
from scipy import stats

data_pvals = stats.norm.sf(data)

<div class='alert alert-warning'>
    <b>ToDo</b> (1 point)
</div>

Compute how many voxels are deemed to be "significant" (assuming $\alpha = 0.05$), using the variable `data_pvals`, in this approach in which we neglect the multiple comparison approach. Store this number (an integer) in a new variable named `nsig_uncorrected`.

In [ ]:
# Implement your ToDo here
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
''' Tests the above ToDo. '''
from niedu.tests.nii.week_6 import test_nsig_uncorrected   
test_nsig_uncorrected(data_pvals, alpha, nsig_uncorrected)

We can create a "mask" by comparing our $p$-values to our significance level and we can give this mask to our plotting-function (`plot_sim_brain`), which will set all "voxels" outside the mask (i.e., those which are `False` in the mask) to 0. 

In [ ]:
smaller_than_alpha = data_pvals < alpha
# Note that 'smaller_than_alpha' is a 2D numpy array with booleans

plot_sim_brain(data, mask=smaller_than_alpha, title=r'Uncorrected ($p < %.4f$)' % alpha)

<div class='alert alert-warning'>
<b>ToDo/ToThink</b> (1 point) 
</div>

Change the value of $\alpha$ (i.e., the variable `alpha`) from before to 0.01. Does the resulting thresholded map look "better"? And what about 0.001? And 0.0001? Theoretically, you could try different values to see what gives the "best" results. This practice of trying out different parameters or strategies leads to another problem: can you think of what this could be? Write down your answer below.

YOUR ANSWER HERE

### 2.4. Bonferroni-correction
Obviously, given that we know our "true effect", we can see that the uncorrected results contain *a lot* of false positives, something that we'd like to avoid! The most obvious way to counter the MCC problem is to adjust the significance level ($\alpha$) by the amount of tests we're performing. Bonferroni correction is such an approach. The way the Bonferroni method does this is by simply dividing the significance level by the amount of tests.

<div class='alert alert-warning'>
<b>ToDo</b> (1 point)
</div>

Calculate the Bonferroni-adjusted significance level (and store this in a variable named `bonf_alpha`) and create a new mask by comparing the previously computed $p$-values against this new significance level. Then, plot the data (using `plot_sim_brain`) with the mask you just created.

In [ ]:
# Implement the ToDo here

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
''' Tests the above ToDo. '''

print("Only hidden tests.")

<div class='alert alert-info'>
<b>ToThink</b> (not graded)
</div>

Many argue that Bonferroni correction for whole-brain fMRI results is too strict (conservative), which is also the case in our simulation (the recovered "blob" is a lot smaller than the true effect).

This conservative nature of Bonferroni correction, for fMRI at least, is due to the violation of a crucial assumption of Bonferroni correctoin. Which assumption is this, and why does fMRI data/results likely violate this assumption?

YOUR ANSWER HERE

### 2.5. FDR correction
As you've seen so far, uncorrected results tend to be too liberal (too many false positives) and Bonferroni-corrected results are too strict (too many false negatives). The "False Discovery Rate-correction" (FDR) technique is a method to adjust $p$-values in a less stringent way. Essentially, while traditional MCC methods (such as Bonferroni) try to control the chance of finding at least one false positive result **amongst all your tests** (i.e. controlling the "familywise error rate" method), the FDR-method tries to limit the proportion of false positives **amongst all your tests which turned out significant**. So, if you set your "FDR-proportion" (confusingly also referred to as "alpha") to 0.05, then it will adjust your initial $p$-values such that out of all your significant results, on average 5% will be false positives.  

In general, FDR-correction is more sensitive than the Bonferroni correction method (i.e. FDR has a lower type 2 error rate/it is less strict), but if you use it, you *do* have to accept that about 5% of your (significant) results are false positives!

Now, let's check out what our results look like after FDR correction:

In [ ]:
from statsmodels.stats.multitest import fdrcorrection
alpha_fdr = 0.05 # we use an alpha of 0.05 (5%)

# The fdrcorrection function already returns a "mask"
# Note that it doesn't accept 2D arrays, so we ravel() and then reshape() it
fdr_mask = fdrcorrection(data_pvals.ravel(), alpha=alpha_fdr)[0]
fdr_mask = fdr_mask.reshape(data.shape)

plot_sim_brain(data, mask=fdr_mask, title='FDR correction')

As you can see, the FDR-correction is way more sensitive than the Bonferroni correction (it "recovers" more of the true signal), but it still results in many false positives (but not as many as uncorrected data).

### 2.6. RFT-based correction
As you've seen in the previous examples, it's quite hard to pick a significance level that strikes a good balance between type 1 errors and type 2 errors, or, phrased differently, between sensitivity (with respect to discovering the true signal) and specificy (i.e. how many of our significant voxels are driven by a true effect).

Let's go back to the results of the Bonferroni correction. We've seen that the results are extremely conservative (few false positives, but many false negatives, i.e. large type 2 error). The major reason for this is that the correction assumes that each test is *independent*, but in our simulation (and in any fMRI dataset), we *know* that there exists spatial correlation, meaning that our tests are *not* independent. In other words, if we know that a certain voxel is signficant in a certain test, it is quite likely that the voxel directly *next* (or above/below) to it is also significant. Therefore, spatially correlated fMRI statistic maps violate Bonferroni's assumption of independent tests (this is also the answer to the ToThink from earlier).

As a possible solution to this problem, neuroscientists have developed a method &mdash; random field theory &mdash; that allows for multiple comparison correction (using FWER) that "corrects" for the smoothness in our data and thresholds accordingly.

Importantly, RFT-correction can either be performed at the voxel-level (testing whether the *amplitude*, i.e., height of the statistic of a voxel is significant, given the smoothness of the data) and at the cluster-level (testing whether the *size* of a cluster of voxels is significantly large, given the smoothness of the data). We'll start with voxel-level RFT.

#### 2.6.1. Voxel-level RFT
Voxel-level RFT allows for "smoothness-adjusted" thresholding for individual voxels. It does so by assuming a particular distribution for the *number of clusters* (or "blobs") one would observe given (1) a particular initial threshold and (2) the smoothness of the data, assuming there is no effect (i.e., the null hypothesis is true). This expected "number of blobs" after thresholding is known as the *Euler characteristic*. And for standard normal data (i.e., $z$-statistics), the expected Euler characteristic is computed as:

\begin{align}
EC = N_{resel}\ (4\ \log_{e}2)\ (2\pi)^{-\frac{2}{3}}\ z\cdot e^{-\frac{1}{2} z}
\end{align}

where $R$ refers to the number of "resels" (a number that depends on the smoothness of your data, which we'll discuss in a bit) and $z$ refers to the $z$-value that you use as an initial threshold. In code, this is:

In [ ]:
def expected_EC(z, n_resel):
    """ Computes the expected Euler Characteristic for a given number of resels
    and initial z-value cutoff. 
    
    Parameters
    ----------
    z : int/float or array of int/float
        Initial z-value cutoff (can be array)
    
    n_resel : int/float
        Number of "resels"
    """
    return n_resel * (4 * np.log(2)) * (2 * np.pi) ** (-(2/3)) * z * np.exp(-0.5 * z ** 2)

Importantly, suppose for now that the number of resels is 1000. Then, we can get the expected number of "blobs" in our data for a given $z$-value threshold, let's say $z = 3$, as follows:

In [ ]:
zthresh = 3
n_blobs = expected_EC(z=zthresh, n_resel=1000)
print("For a z-threshold of %i, we expect %.2f blobs in random 2D data with 100 resels." % (zthresh, n_blobs))

We can also evaluate the expected EC for a range of potential $z$-value thresholds (e.g., from 0-5) and plot it:

In [ ]:
zx = np.linspace(0, 5, 100)  # 100 values between 0 and 5
ecs = expected_EC(zx, n_resel=1000)  # expected EC also works for multiple z-values at once

In [ ]:
plt.figure(figsize=(10, 4))
plt.plot(zx, ecs)
plt.ylabel('Expected EC', fontsize=20)
plt.xlabel('Z-value threshold', fontsize=20)
plt.grid()

To compute the Euler characteristic, we first need to know how to estimate the number of "resels" for our data. You can think of the number of resels as the number of truly independent elements in your data ("resel" is short for "RESolution ELement"). The number of resels is usually estimated by dividing the number of voxels by the estimated size of the resel. For our simulated 2D data, the number of resels is defined as follows:

\begin{align}
N_{resel} = \frac{N_{X}\cdot N_{Y}}{\mathrm{size}_{resel}}
\end{align}

where $N_{X}$ is the number of "voxels" in the first dimension and $N_{Y}$ the number of "voxels" in the second dimension, and where the resel size ($\mathrm{size}_{resel}$) is estimated as the product of the smoothness of our data in all dimensions, measured in FWHM:

\begin{align}
\mathrm{size}_{resel} = \mathrm{FWHM}_{X} \cdot \mathrm{FWHM}_{Y}
\end{align}

So, given a particular size of our resel, $N_{resel}$ represents how many resels there would "fit" in our data.

<div class='alert alert-warning'>
    <b>ToDo</b> (1 point)
</div>

Usually, the smoothness of the data has to be estimated (usually from the residuals), but in our simulation, we know the smoothness: it's the FWHM we used for our gaussian filter to smooth out data! Compute the number of resels in our simulated data and store it in a variable named `n_resel`.

In [ ]:
# Implement your ToDo here

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
''' Tests the above ToDo. '''
from niedu.tests.nii.week_6 import test_n_resel
test_n_resel(data, n_resel)

Now, another way to interpret EC values is as $p$-values: the chance of finding one or more "blobs" for a given $z$-value! This way, we can choose a particular $z$-value threshold that would correspond to $p = 0.05$. We do this below:

In [ ]:
ecs = expected_EC(zx, n_resel)

# find the index of the EC value closest to alpha
idx_z = np.abs(ecs - alpha).argmin()

# Index the z-values with idx_z
z_thresh = zx[idx_z]
print("The z-value threshold corresponding to p = 0.05: %.3f" % z_thresh)

<div class='alert alert-info'>
<b>ToThink</b> (1 point)
</div>

As you (should) see in the plot above, the RTF-based correction is still quite strict/conservative (i.e. misses quite some of the true effect), although arguably not as strict as Bonferroni correction. Given the way how to calculate the number of RESELS, can you think of two ways on how to improve the sensitivity of RFT-based MCC?

YOUR ANSWER HERE

### 2.6.2 Cluster-level RFT
In all the previous MCC techniques, we have used voxel-level corrections, which resulting $p$-values tell us something about whether the *height* of a voxel's statistic (often referred to as "amplitude") is higher than would be expected under the null-hypothesis. Basically, because we investigated *per voxel* whether its value is higher than expected, we are making inferences on the level of voxels. 

Another type of inference is *cluster*-level inference, in which you do not test the voxel amplitude, but the *size of clusters*. Basically, in this type of cluster-extent testing, you are investigating whether the size of the clusters you find are (significantly) larger than to be expected under the null-hypothesis (i.e., no effect). 

However, as you can imagine, the null-distribution of cluster sizes (i.e. the size of "significant" clusters you'd expect by chance alone) depends strongly on the initial smoothness of your data. Again: RFT to the rescue! 

Basically, RFT can *also* give us the $p$-value for clusters, given their size, by estimating the null-distribution of cluster-sizes based on the data's smoothness. So, instead of giving us the $p$-value for voxels based on the height of their value and the data's smoothness (i.e., voxel-level RFT), RFT can also do this on the *cluster-level* by investigating the $p$-value of the size of clusters. See how these two RFT-methods relate to each other? They're doing the same thing &mdash; estimating a null-distribution given the smoothness of the data &mdash; but for different things: either for the *height* of the ($z$-)statistic value per voxel (voxel-level RFT) or for the *size* per cluster (cluster-level RFT). 

How RFT does this is way beyond the scope of this course, but we'll walk you through it conceptually, so that you understand the implications of this technique.

Anyway, a first step in cluster-level RFT is to determine a minimum (cutoff) value for your statistics map, which you can use to evaluate whether there are actually clusters in your data. Let's look at an example, in which we use a minimum value of 3.0:

In [ ]:
min_z = 3
thresh_data = (data > min_z)

plot_sim_brain(data, mask=thresh_data, title=r"Clusters after thresholding at $z$ > 3")

Now, we can use cluster-based RFT to calculate the $p$-value for each cluster in the above thresholded data plot. This $p$-value reflects the probably of this cluster-size (or larger) under the null-hypothesis. We can then threshold this map with clusters, using a 'cluster-wise' $p$-value cutoff of 0.01 for example, and plot it again to see how this method affects type 1 and type 2 errors. The function below (`threshold_RFT_cluster`) takes three arguments: the statistics-map (our `data` variable), a minimum $z$-value, and a $p$-value cutoff which is used to threshold the clusters.

Below, we do this for a $z$-threshold of 3.1 (corresponding to a $p$-value of approx. 0.001) and a cluster $p$-value threshold of 0.05.

In [ ]:
from niedu.utils.nii import rft_cluster_threshold

rft_cl_mask = rft_cluster_threshold(data, z_thresh=3.1, p_clust=0.01)
plot_sim_brain(data, mask=rft_cl_mask, title='RFT thresholding (cluster-based)')

From the above plots, you should see that cluster-thresholding can be a very sensitive way to threshold your data if you expect your effects to occur in relatively large clusters (and given that you're able to estimate the smoothness of the data appropriately, something that is a topic of debate). As such, it is by far the most used MCC method in univariate fMRI research today. As part of a the next couple of ToDos/ToThinks, you'll get a new statistics image and you have to argue why cluster-based RFT thresholding is appropriate or not.

### 2.7. Non-parametric MCC
In addition to the previously discussed MCC approaches (which are common in parametric group-level models), non-parametric analyses offer another approach. In this approach, the algorithm keeps track of the *maximum* statistic across permutations. This statistic can refer to the highest voxel-wise amplitude (for voxel-based), largest cluster size (for cluster-based, given some initial $z$-value cutoff), or even highest TFCE-transformed amplitude, across permutations.

If we, for example, want to perform a cluster-based non-parametric analyses, we can save the largest cluster size (given some initial $z$-value threshold) for each iteration. Then, across our (let's say) 5000 permutations, we have acquired a *distribution* of maximum cluster sizes under the null hypothesis of no effect. 

We actually did this for our simulated data: we kept track of the maximum cluster size across 1000 permutations given some initial $z$-value cutoff. We'll plot such a non-parametric distribution below (for an arbitrary $z$-value cutoff of 3):

In [ ]:
np_dist = np.load('clust_size_dist_data.npz')
zx, clust_sizes = np_dist['zx'], np_dist['dist']
z_cutoff = 3
z_idx = np.abs(zx - z_cutoff).argmin()
clust_size_dist = clust_sizes[:, z_idx]

plt.figure(figsize=(15, 5))
plt.title("Max. cluster size across 1000 permutation", fontsize=25)
plt.hist(clust_size_dist, bins=50)
plt.xlabel("Max. cluster size", fontsize=20)
plt.ylabel("Frequency", fontsize=20)
plt.grid()
plt.show()

With that information, we can calculate the non-parametric $p$-value of each of our *observed* clusters using the same type of formula as we used earlier:

\begin{align}
p_{\mathrm{cluster}} = \frac{\sum_{i=1}^{P}\mathbf{I}(\mathbf{\mathrm{max.\ cluster\ size}}^{perm}_{i} \geq \mathrm{cluster\ size}^{obs}) + 1}{P + 1}
\end{align}

The same logic holds for voxel-based (TFCE-transformed) amplitude, where you wouldn't keep track of the maximum cluster size, but the maximum amplitude across permutations.

<div class='alert alert-warning'>
    <b>ToDo</b> (ungraded/optional)
</div> 

Suppose I have found an effect within our simulated data with a cluster size of 231 "voxels" (using an initial $z$-value threshold of 3). Using the distribution of maximum cluster sizes above (i.e., the variable `clust_size_dist`), can you compute the associated cluster $p$-value? Store it in a variable named `pval_clust_size`.

In [ ]:
# Implement the (optional) ToDo here

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
''' Tests the above (optional) ToDo. '''
np.testing.assert_almost_equal(pval_clust_size, 0.001998)
print("Well done!")

In our experience, non-parametric analyses (e.g., `randomise` in FSL) in combination with TFCE (also supported in FSL) is a very sensitive approach, allowing for voxel-wise inference while taking into account the "blobbiness" of effects!

### 2.7. Exercise on new data
Suppose we repeat the cat-picture experiment which we described earlier. Based on the literature, we expect to find strong activation in a small group of voxels &mdash; known as the *nucleus felix* &mdash; which is about 29 "voxels" in volume, located in the middle of the brain (here: our 2D brain). Like our other example, we've measured a group-level (2D) statistics ($z$-values) map which represents the cat-against-baseline contrast.

We'll load in and plot the new data below:

In [ ]:
data2 = np.load('data_assignment.npy')
plot_sim_brain(data2, title='Simulated data assignment', vmin=-10, vmax=10)

<div class='alert alert-warning'>
<b>ToDo/ToThink</b> (1 point)
</div>

Given that cluster-based RFT correction worked really well in our last example, should we use this technique again on this dataset, given our expectations of the true effect?  Why (not)? Hint: actually apply the cluster-based RFT correction to the new data (you may assume that the new data has the same smoothness as the previous data).

In [ ]:
# Apply cluster-based RFT


YOUR ANSWER HERE

### 2.8. Effect of different MCC strategies on real data
We actually ran different group-level analyses (using FLAME1-type mixed-effects) on our run-level $4\cdot \beta_{face} - \beta_{place} - \beta_{body} - \beta_{character} - \beta_{object}$ contrast from 12 subjects, which are located here:

In [ ]:
import os
from glob import glob

gl_paths = sorted(glob(os.path.join(fsl_deriv_dir, 'grouplevel_task-flocBLOCKED', '*')))
print('\n'.join(gl_paths))

As you can see, there are results for three different MCC strategies:
* uncorrected (with $p < 0.05$);
* cluster-based (with $z > 3.1$ and $p_{\mathrm{cluster}} > 0.05$);
* voxel-based RFT (with $p_{\mathrm{voxel}} > 0.05$);
* non-parametric ("randomise" with TFCE, non-parametric $p < 0.05$)

Let's take a look at the thresholded $z$-statistic maps for each of those analyses.

<div class='alert alert-warning'>
    <b>ToDo</b> (ungraded)
</div>

If not open yet, open FSLeyes. Add the standard MNI brain as a background image (`File` &rarr; `Add standard` &rarr; select `MNI152_T1_2mm_brain.nii.gz`). Then, add the `thresh_zstat1.nii.gz` image from the `contrast-faceGTother_method-FLAME1_thresh-uncorr05.gfeat/cope1.feat` directory (`File` &rarr; `Add from file`). Change the colormap to "Red-Yellow".

You have seen this brain map before, and you should know by now that this brain map likely contains many false positives as it's not corrected for multiple comparisons. 

<div class='alert alert-warning'>
    <b>ToDo</b> (ungraded)
</div>

Add the `thresh_zstat1.nii.gz` file from the `contrast-faceGTother_method-FLAME1_thresh-cluster.gfeat/cope1.feat` directory (`File` &rarr; `Add from file`). Change to colormap to "Blue-Light blue". 

Here, you see a much more modest effect, where only a couple of clusters (in the superior temporal gyrus and posterior cingulate cortex) survived.

<div class='alert alert-warning'>
    <b>ToDo</b> (ungraded)
</div>

Add the `thresh_zstat1.nii.gz` file from the `contrast-faceGTother_method-FLAME1_thresh-voxel.gfeat/cope1.feat` directory, and change the colormap to "Green".

If you don't see any green voxels, that's right! No voxel "survived" the relatively conservative voxel-based RFT tresholding!

<div class='alert alert-warning'>
    <b>ToDo</b> (ungraded)
</div>

Lastly, add the `thresh_zstat1.nii.gz` file from the `contrast-faceGTother_method-randmoise_thresh-TFCE05.gfeat/cope1.feat` directory, and change the colormap to "Blue".

This looks quite alright (in the sense that at least some voxels survive the MCC procedure)! Does this mean that we should always use cluster-based or non-parametric (TFCE-boosted) MCC? Not necessarily. Like always, this depends on your data, the effect you expect, and the conclusions that you want to draw from your results.

## 3. ROI-based analysis
In the previous sections, we've discussed how to implement "whole-brain" analyses: models that are applied to *all* voxels in the brain separately. As we've seen in the previous section, this rather "exploratory" approach leads to issues of multiple comparisons, which are deceptively difficult to properly control for.

One alternative way to deal with the multiple comparisons problem is a more "confirmatory" approach: region-of-interest (ROI) analysis, which is the topic of this section.

### 3.1. What is an ROI?
An ROI is a prespecified brain region that is hypothesized to contain the hypothesized effects. These can be anatomically or functionally defined. Anatomical ROIs are regions of interest based on anatomical landmarks (such as folding pattern or histology), for example the caudate nucleus, inferior temporal gyrus, and anterior cingulate cortex.

As opposed to anatomical ROIs, functional ROIs are not based on anatomical properties, but on *functional* properties. These regions are usually determined on other, independent data and analyses. Sometimes, the data used for defining such a functional ROI comes from the same study, which acquired a so-called "functional localizer" in order to define a ROI for analysis of other data. This is also what we did in the NI-edu dataset: we acquired the `floc` runs in order to define functionally-defined "FFA" regions (based on the $\beta_{face} > \mathrm{other}$ contrast) that can be used for ROI-analyses of the `face perception` runs (e.g., to answer questions like: does attractiveness of faces modulate FFA activity?).

In this section, we'll show you how a typical ROI-analysis is done using an anatomical ROI. Then, the section after that will discuss how to derive functional ROIs within a single dataset.

### 3.2. An example ROI-analysis using an anatomical ROI
For our anatomical ROI-analysis, we'll stick to the `flocBLOCKED` data. Specifically, we will analyze the "face > other" contrast (i.e., $4\cdot \beta_{face} - \beta_{place} - \beta_{body} - \beta_{character} - \beta_{object}$) within the right amygdala (similar to [this study](https://academic.oup.com/scan/article/3/4/303/1628260)). Our group-level hypothesis is that the amygdala activates significantly in response to faces (i.e., a one-sample, one-sided $t$-test): 

* $H_{0}: \beta^{\dagger}_{face > other} = 0$
* $H_{a}: \beta^{\dagger}_{face > other} > 0$

The data that we load in below represents run-level $c\hat{\beta}^{*}$ images corresponding to the "face > other" contrast from our 12 subjects. Data is in MNI152 (2 mm) space with voxel dimensions $[91, 109, 91]$.

In [ ]:
import nibabel as nib
cope_4d_img = nib.load('contrast-faceGTother_level-run_stat-cope.nii.gz')
cope_4d = cope_4d_img.get_fdata()
print("Shape of data: %s" % (cope_4d.shape,))

Let's visualize this data. We'll plot (using `plt.imshow`) a single (right-hemisphere) sagittal slice of each subjects' COPE-image, with red indicating positive contrast-values (i.e., $4\cdot \beta_{face} - \beta_{place} - \beta_{body} - \beta_{character} - \beta_{object} > 0$) and blue indicating negative contrast-values (i.e., $4\cdot \beta_{face} - \beta_{place} - \beta_{body} - \beta_{character} - \beta_{object} < 0$).

In [ ]:
from nilearn.datasets import load_mni152_template
mni = load_mni152_template().get_fdata()

plt.figure(figsize=(12, 6))
subjects = ['sub-%s' % str(i).zfill(2) for i in range(2, 14)] 

for i in range(cope_4d.shape[-1]):
    plt.subplot(2, 6, i+1)
    plt.title(subjects[i], fontsize=20)
    plt.imshow(mni[:, :, 26].T, origin='lower', cmap='gray')
    this_data = cope_4d[:, :, 26, i]
    plt.imshow(this_data.T, cmap='seismic', origin='lower',
               vmin=-500, vmax=500, alpha=0.6)
    plt.axis('off')
plt.tight_layout()
plt.show()

Now, conventional (exploratory) whole-brain analysis would fit a GLM on each voxel separately, but for our ROI-analysis, we're going to do it differently: we're only going to analyze the across-voxel average lower-level $c\hat{\beta}^{*}$ values within the right amygdala.

We'll load in the right amygdala mask first. This is a region from the probabilistic Harvard-Oxford subcortical atlas.

In [ ]:
amygdala_probmask_img = nib.load('amygdala_lat-right_atlas-harvardoxford_thresh-0.nii.gz')
amygdala_probmask = amygdala_probmask_img.get_fdata()
print("Shape of probabilistic amygdala mask: %s" % (amygdala_probmask.shape,))

<div class='alert alert-danger'>
<b>Assignment</b> (3 points)
</div>

We now have the data (of shape $91 \times 109 \times 91 \times 12$) and a mask (of shape $91 \times 109 \times 91$). In this assignment, you'll implement the actual ROI analysis using this data.

You have to do the following:
1. Create a boolean mask (array with `True`/`False` values) by tresholding the probabilitic mask at 20 (i.e., $p(\mathrm{amygdala}) \geq 0.2$). There should be 549 voxels surviving this threshold;
2. Average, per subject, the $c\hat{\beta}^{*}$ values within this mask. You *do not* need a for-loop for this (hint: numpy broadcasting!);
3. Set up an appropriate group-level design matrix ($\mathbf{X}^{\dagger}$) that allows you to compute the average effect ($\beta^{\dagger}$) across the 12 ROI-average values ($y^{\dagger}$) and run a GLM;
4. With an appropriate group-level contrast, compute the $t$-value and $p$-value associated with our group-level hypothesis (outlined in the beginning of this section).

Store the ROI-average values (in total 12) as a numpy array in a variable named `y_group`, your group-level design matrix (a $12 \times 1$ numpy array) in a variable named `X_group`, your group-level parameter (there should be only one) in a variabled named `beta_group`, and your $t$-value and $p$-value in variables named `t_group` and `p_group` respectively. You can use the `stats.t.sf` function to calculate the *one-sided* $p$-value from your $t$-value.

In [ ]:
# Implement step 1 and 2 here (y_group)

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
''' Tests step 1 and 2 (hidden tests only). '''

In [ ]:
# Implement step 3 (X_group and beta_group)
from numpy.linalg import inv

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
''' Tests step 3 (hidden test only). '''


In [ ]:
# Implement step 4 here (t and p-value)
from scipy import stats

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
''' Tests step 4 (t_group, p_group). '''

### 3.3. Functional ROIs & neurosynth
In addition to anatomical ROIs, you can also restrict your analyses to functionally defined ROIs. "Functionally defined", here, means that the region has been derived from or determined by its assumed function. This can be done using a (separate) dataset specifically designed to localize a particular functional region (the "functional localizer" approach). Another option is to define your ROI based on a meta-analysis on a particular topic. For example, you could base your ROI on (a set of) region(s) on a meta-analysis of the neural correlates of face processing.

One particular uesful tool for this is [neurosynth](https://neurosynth.org/) (see also the associated [article](https://www.nature.com/articles/nmeth.1635)). Neurosynth is a tool that, amongst other things, can perform "automatic" meta-analyses of fMRI data. As stated on its [website](https://neurosynth.org/), "it takes thousands of published articles reporting the results of fMRI studies, chews on them for a bit, and then spits out images" that you can use, for example, a functional ROIs.

<div class='alert alert-warning'>
    <b>ToDo</b> (ungraded)
</div>

Go to [https://neurosynth.org](https://neurosynth.org/) and read through the text on its homepage. Then, click on "Meta-analyses" &rarr; "Terms". Here, you can search for a particular term that you want Neurosynth to compute the meta-analysis for. For example, in the search bar on the right, fill in the term "face", and you should see a list of "face" related terms (e.g., "emotional faces", "face", "face ffa", "face recognition", etc.). 

Now, click on the term "emotional faces".

After selecting a particular term, Neurosynth will open a new page with an interactive image viewer. By default, it shows you the result (i.e., $z$-values) of an "association test" of your selected term across voxels, which tests whether voxels are *preferentially* related to your selected term (click on "FAQs" for more information). 

<div class='alert alert-warning'>
    <b>ToDo</b> (ungraded)
</div>

Under "Thresholds", move the right slider up and down to adjust the $z$-value threshold of the association test values, which should immediately update the images on the right. 

<div class='alert alert-warning'>
    <b>ToDo</b> (ungraded)
</div>

Click on "Studies" to view the studies that Neurosynth based this meta-analysis on. You'll see that the exact topic differs across studies (from face processing in alcohol use disorder to the role of expectations in face processing), but the term ("face processing") is common to all studies.

<div class='alert alert-warning'>
    <b>ToDo</b> (1 point)
</div>

Now, suppose that I believe that perceiving faces is intrisically rewarding, because humans are very much social animals, and I want to see whether perceiving faces (relative to other types of images) significantly activates the brain's reward circuitry. 

Conduct a meta-analysis using the term "reward", set the threshold to $z > 12$ (note that this exact value is somewhat arbitrary; make sure to decide on this cutoff *before* looking at your own data as to avoid circularity in your analysis!). Download this thresholded map by clicking on the download symbol (next to the trash bin icon) next to "reward: association test". This will download a nifti file *to your own computer*.

For students at the UvA, upload this image to the server as follows:
* Under the "Files" tab in Jupyterhub, navigate to your `week_6` folder (with your mouse);
* Click on the "Upload" button, select the downloaded file (`reward_association-test_z_FDR_0.01.nii.gz`), and click on the blue "Upload" button to confirm.

For non-UvA students, move this file to your own `week_6` directory.

In [ ]:
''' Tests the ToDo above. '''
import os.path as op
here = op.abspath('')
par_dir = op.basename(op.dirname(here))

if par_dir != 'source':  # must be student account
    try:
        assert(op.isfile('reward_association-test_z_FDR_0.01.nii.gz'))
    except AssertionError:
        msg = "Could not find 'reward_association-test_z_FDR_0.01.nii.gz' file!"
        raise ValueError(msg)
    else:
        print("Well done!")

<div class='alert alert-warning'>
    <b>ToDo</b> (ungraded/optional)
</div>

Although slightly off-topic, another great Neurosynth functionality is its "decoder". As opposed to its meta-analysis functionality which generates the most likely (and specific) brain map given a particular term, the decoder produces the most likely terms given a particular (untresholded) brain map! This is great way to help you interpret your (whole-brain) results (but this does *not* replace a proper and rigorous experimental design).

To use the decoder, click on the "Decoder" tab on the Neurosynth website and subsequently on the "Upload an Image" button. This will redirect you to another website, [Neurovault](https://neurovault.org/). You'll have to create an account on Neurovault in order to use Neurosynth's decoder (if you don't want to, that's fine, as this is an optional ToDo).

In this week's directory (`week_6`), we included the unthresholded $z$-value map from the group-level (mixed-effects) analysis of the $4\cdot \beta_{face} - \beta_{place} - \beta_{body} - \beta_{character} - \beta_{object}$ contrast: `contrast-faceGTother_level-group_stat-zstat.nii.gz`. You need to download this file to your computer, which you can do by navigating to your `week_6` directory in Jupyterhub, selecting the file, and clicking on "Download". Now, let's decode this image.

After registration on the Neurovault website, you should be redirected to a short "questionnaire" where you have to fill in some metadata associated with the image. Give it a name (e.g., "FFA localizer"), add it to a particular "collection" ("{your username} temporary collection" is fine), set the map type to "Z map", set the modality & acquisition type to "fMRI-BOLD", and "Cognitive Atlas Paradigm" to "face recognition task" (this is unimportant for now). Finally, under "File with the unthresholded volume map", select the just downloaded file and click "Submit" at the very bottom.

You should be redirected to "Neurosynth", which should now visualize our group-level "face > other" $z$-statistic file. Now, at the bottom of the page, you find a table with "Term similarity", showing the terms whose unthresholded brain maps correlate the most with our just uploaded brain map. Scrolling through these terms, you'll notice a lot of terms associated with the "default mode network" (DMN)! We are not sure why our "face map" correlates so highly with the brain map typically associated with the DMN ... One reason might be that faces deactivate the DMN *less* than the other conditions (bodies, places, etc.), which shows up as a positive effect in the "face > other" contrast. 

---

Alright, let's get back to the topic of functional ROIs again. Given that you defined a functional ROI, the process of analyzing your data proceeds in the same fashion as with anatomical ROIs:
1. You average your lower-level data ($c\hat{\beta}^{*}$) within your ROI;
2. You construct a group-level design matrix and run a GLM;
3. Proceed as usual (construct group-level contrats, calculate $t$/$z$-values, etc.)

### The end!
You finished the last (mandatory) notebook! If you want, there is an optional lab next week, in which you can try out the functionality of the `Nilearn` package.

In [ ]:
print("Well done!")